In [27]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

# 数据生成
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)  # 生成随机噪声数据
input_num = features.shape[1]
output_num = labels.shape[1]

In [28]:
help(d2l.synthetic_data)

Help on function synthetic_data in module d2l.torch:

synthetic_data(w, b, num_examples)
    Generate y = Xw + b + noise.
    
    Defined in :numref:`sec_linear_scratch`



In [29]:
print(features.shape)
print(labels.shape)

torch.Size([1000, 2])
torch.Size([1000, 1])


## 读取数据集

In [30]:
batch_size = 10

dataset = torch.utils.data.TensorDataset(features, labels)  # 可索引
dataload = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)  # 不可索引

## 网络模型

In [31]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, input_num, output_num):
        super().__init__()
        self.hidden = nn.Linear(input_num, output_num)
    
    def forward(self, x):
        x = self.hidden(x)
        return x

In [65]:
# 初始化权重和偏置
net.hidden.weight.data.normal_(mean=0, std=0.01)  # 正态分布填充
net.hidden.bias.data.fill_(0)  # 零填充

tensor([[0.]])

## 损失函数

In [66]:
# 均方误差
loss = nn.MSELoss()

## 优化器

In [67]:
# 小批量随机梯度下降
lr = 0.03
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

## 训练

In [74]:
epochs = 3
for epoch in range(epochs):
    for X, y in dataload:  # 真实数据
        l = loss(net(X), y)
        optimizer.zero_grad()  # 反向传播前要清空梯度
        l.backward()  # 反向传播
        optimizer.step()  # 梯度更新
    # 验证
    l = loss(net(features), labels)
    print('epoch {}, loss {}'.format(epoch + 1, l))

epoch 1, loss 0.0001037598995026201
epoch 2, loss 0.00010369334631832317
epoch 3, loss 0.00010340638255001977


In [75]:
w = net.hidden.weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net.hidden.bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0002, -0.0005])
b的估计误差： tensor([[0.0003]])
